# EDA THOMAS

**Intent**
I want to take the agreed upon dataset [spider 1.0's soccer_1 set](https://www.kaggle.com/datasets/jeromeblanchet/yale-universitys-spider-10-nlp-dataset/code)
This requires downloading the zip and searching for soccer_1. 
soccer_1 is choosen due to the db being the greatest size. It's a couple hundred megabytes.

**Preliminary Setup**
I'll first take the soccer_1.sqlite file and hook it up to this notebook so that we can do some observations. Since the data is already loaded into the database, this should work as is.
If something goes wrong, the schema.sql contains the ddl necessary to rebuild the database.

I am not super familiar with soccer, so a bunch of this is going to be guesing/understanding the schema, which is a process we'll have to do anyways
Originally, I wanted to use a python 3.12 virtual env. for this however, the model I chose to work with is officially compatible with python 3.8.
Python 3.8 is no longer supported as of last year. Because of this, downloading it on windows is a bit more complicated. As such, I'll use 3.9 instead.

I was not aware of this, but sqlite3 is [shipped by default with python](https://www.tutorialspoint.com/sqlite/sqlite_python.htm).

In [1]:
# preliminary setup/helpful functions

import sqlite3
import pandas

FETCH_TYPE_ONE = "one"
FETCH_TYPE_ALL = "all"

def run_command(command, fetch_type=FETCH_TYPE_ONE):
    
    cursor = sqlite3.connect("database/soccer_1.sqlite")
    
    result = None
    if fetch_type == FETCH_TYPE_ONE:
        result = cursor.execute(command).fetchone()
    elif fetch_type == FETCH_TYPE_ALL:
        result = cursor.execute(command).fetchall()                
        
    cursor.close()
    return result

# testing the connection by printing out the amount of records in the player table
run_command("SELECT COUNT(*) FROM PLAYER")

(11060,)

**Schema Observations**
With the database now connected, we can do a brief overview about the data. This will help us figure out what questions we can ask our chosen model.

In [5]:
# First we can observe what other tables our schema has.
tables = run_command("SELECT tbl_name FROM sqlite_master WHERE type = 'table'", FETCH_TYPE_ALL)

# Looking at the table names, it looks like sqlite_sequence is a table that we will not be interested in using. 
# With that, we can quickly filter it out and look at the columns of all the other tables
parsed_tables = []
for table_tuple in tables:
    table_name = table_tuple[0]
    if table_name != "sqlite_sequence":
        parsed_tables.append(table_name)

player_table = 'Player'
player_attributes_table = 'Player_Attributes'

# first we'll print the table names
print(parsed_tables)

['Player_Attributes', 'Player', 'League', 'Country', 'Team', 'Team_Attributes']


From these table names, We can see a couple of relations start to form.
1. We know that a player belongs to a team and that a team has attributes.
2. A player also has attributes. The mapping between a player and their attributes is done by a composite foreign key, on api_id and fifa_api_id

But how do leagues, countries, and teams relate to one another?
We could load the columns for each of those three tables and figure out which ones have common columns.

Following [this guide](https://www.daniweb.com/programming/software-development/threads/124403/sqlite3-how-to-see-column-names-for-table), we can use the description from the cursor upon selecting a table to figure out
what columns that database contains.

In [6]:
tables_and_columns = {}
tmp_cursor = sqlite3.connect("database/soccer_1.sqlite")
for name in parsed_tables:
    command = tmp_cursor.execute(f"SELECT * FROM {name}")
    col_name_list = [tuple[0] for tuple in command.description]
    
    tables_and_columns[name] = col_name_list
tmp_cursor.close()

# looking at the tables and columns object as a whole, we can say with confidence that the player attributes table
# contains the most data
print(f"The Player_Attributes table contains: {len(tables_and_columns[player_attributes_table])} columns.")

# how many of these are numerical and do not pertain to an id?
player_attributes_fields: list[str] = tables_and_columns[player_attributes_table]

# to figure this out, we can rule out all id related fields. for simpliticty sake we will take the first row of the table as well
first_row = run_command(f"SELECT * FROM {player_attributes_table}")

# generate a truth list and iterate through it while also attempting to convert the matching row to an int
non_id_fields = list(map(lambda column_name: True if 'id' not in column_name.lower() else False, player_attributes_fields))

numerical_nonid_rows = []
for i in range(len(first_row)):
    if non_id_fields[i]:
        try:
            int(first_row[i])
            numerical_nonid_rows.append(non_id_fields[i])
        except: ValueError

print(f"The Player_Attributes table contains: {len(numerical_nonid_rows)} columns that are numerical and not tied to an id.")

The Player_Attributes table contains: 42 columns.
The Player_Attributes table contains: 34 columns that are numerical and not tied to an id.


With the columns for each table present, figuring out what relates to what is much easier. I've also took some extra effort to understand some basics in Fifa so that I could make better sense of the data.

The **Player_Attributes** table directly maps to the **Player** table using the **player_fifa_api_id** and **player_api_id**.

The **Player** table provides the player's name, height, and birthday.
The **Player_Attributes** table includes multiple stats for a given player. These are either categorical (your preferred foot is either left or right), or numerical as a number out of 100. 

Potential is the maximum your overall rating can reach, which is a number. I'll admit that I do not fully understand how this number is computed, so I'll likely not use it.

With all of this in mind, I have a couple of goals. First being that I want to answer questions related to a singular table. More specifically the player table. Who is the heaviest player? Who is the tallest player? The shortest? The oldest? The youngest?
Afterwards, I want to ask questions related to both the player and player attributes table. Answers should use the player's name, even if the player's name is in a table separate from the player's attributes.

Below, I'll include a couple of the queries I am talking about

In [5]:
result_tallest = run_command(f"SELECT player_name, MAX(height) FROM {player_table}")

# 208.28 cm is 6 feet, 9.89 inches, which is impressively tall.
print(result_tallest)

('Kristof van Hout', 208.28)


In [6]:
result_oldest = run_command(f"SELECT player_name, MIN(birthday) FROM {player_table}")

# I dont't have the year of this dataset, but it is likely sometime in the 2020s. Sports careers are taxing
# and do not last as typically long as other careers. Assuming the data is from 2020, they would be
# about 87 at the time the data was created. Googling who they are, it looks like they retired at the age of 42.
print(result_oldest)

('Alberto Fontana', '1967-01-23 00:00:00')


In [ ]:
result_preferred_foot = run_command(f"SELECT preferred_foot, COUNT(id) FROM {player_attributes_table} GROUP BY preferred_foot", FETCH_TYPE_ALL)

print(result_preferred_foot)

[(None, 836), ('left', 44733), ('right', 138409)]


In [7]:
highest_shot_power = run_command(
    f"""
        SELECT p.player_name, max(a.shot_power) FROM {player_table} p 
            LEFT JOIN {player_attributes_table} a
                on p.player_api_id = a.player_api_id AND p.player_fifa_api_id = a.player_fifa_api_id
    """
)

print(highest_shot_power)

('John Arne Riise', 97)


**Model Selection and Testing**
Using https://huggingface.co/, when it comes to model selection for table question answering, there are two primary developments that standout to me. The first being Google and the second being Microsoft. I decided to give the Microsoft model a shot. https://huggingface.co/microsoft/tapex-base-finetuned-wtq

Microsoft's model focuses on TAPEX, which by the model description stands for "Table Pre-training via Execution".
It took some digging around to figure out what libraries are being used. Turns out that either flax, pytorch, or tensorflow is a necessary backend, while the transformers library is provided here. https://github.com/huggingface/transformers

In [ ]:
# The code from this module is pretty much a full on copy and paste of the code on: https://huggingface.co/microsoft/tapex-base-finetuned-wtq.
# I did this because I wanted to ensure that the model will still execute with the instructions provided.

from transformers import TapexTokenizer, BartForConditionalGeneration

tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base-finetuned-wtq")

data = {
    "year": [1896, 1900, 1904, 2004, 2008, 2012],
    "city": ["athens", "paris", "st. louis", "athens", "beijing", "london"]
}

table = pandas.DataFrame.from_dict(data)
query = "In which year did beijing host the Olympic Games?"
encoding = tokenizer(table=table, query=query, return_tensors="pt")

outputs = model.generate(**encoding)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

c:\Users\spenc\DSCI-2025-TEAM-A\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[' 2008']


In [ ]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base-finetuned-wtq")

# for applying the model the data i've acquired, I chose to start off with a subset of the current data
player_and_height = run_command(f"SELECT player_name, height FROM {player_table}", FETCH_TYPE_ALL)

# load the data into a pandas dataframe and then pass that as part of the encoding
df = pandas.DataFrame.from_records(data=player_and_height, columns=["Player Name", "Height"])

# since all columns are meant to be strings here, we can convert them likeso.
# tapex handles ints and strs. no other data type yet.
df = df.astype(str)

# There is also a limit that the tokenizer can take. It might have to do with pytorch?
query = "What player name is 208.28 Height?"
encoding = tokenizer(table=df, query=query, return_tensors="pt", truncation=True)

outputs = model.generate(**encoding)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


query="What player has the maximum height?"
encoding = tokenizer(table=df, query=query, return_tensors="pt", truncation=True)

# by enabling truncation, this problem is fixed, but.. the answer isn't accurate
# i'm not even sure how to even ask the question?
outputs = model.generate(**encoding)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# same answer is spit out, so not necessarily answering the question


[' abdoulaye diallo']
[' abdoulaye diallo sadio,22']


**Further Exploration**

I figured that the random answers here are due to the fact that we haven't pre-trained the model with our data. We need to "synthesize" our own pre-training data as pointed out here: https://github.com/microsoft/Table-Pretraining. To begin with, I downloaded this repo and tried to figure out how synthesizing works.

Due to potential usage laws, I can likely share the general training data format, and what I've done to train it (if possible) but nothing else.

Even with a venv using python 3.9, I still ran into a conflict with importlib-metadata==4.8.1 because the juypter notebook requires importlib-metadata>=4.8.3. To work around this, I commented out the 4.8.1 portion in the requirements.txt of the tapex training repo in hopes that no compatibility issues would occur. This seemed to have workled.

Running just main.py within the data_generator with default parameters reached 21% before estimating a > 2 hour runtime. I never finished it. The instructions on how to generate your own "testing/training" data are extremely vague, so this is as far as I went. Further study involving taking part of the tapex training dataset and isolating it on a singular schema basis would provide value. With that knowledge I could try to make my own training data for the dataset I wanted to test out. I'm not sure how I'd "train" the model with it, though

Additionally, the tokenizer it's using is: tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large"). is it possible for us to swap this to use microsft's implementation instead? (I recall TAPEX is built off of facebook's bart), but it is a bit strange to see that pretraining repo is not using tapex. This might be okay actually. bart-large is likely what provides the language understanding layer.